In [245]:
import torch
from PIL import Image
import os
import re
from settings import *
import torch
import numpy as np


In [13]:
torch.manual_seed(0)
torch.cuda.is_available()
torch.backends.mps.is_available()
device = torch.device('cpu')

In [273]:
from PIL import Image
import os
external_image_path = "test.jpg"
folder_path = "images/happy/"
MIN_SIZE = 256
sizes = []
for filename in os.listdir(folder_path):
    img = Image.open(os.path.join(folder_path, filename))
    sizes.append(img.size)
if sizes:
    min_width, min_height = min(sizes)
    new_external_size = (min(min_width, min_height), min(min_width, min_height))
    new_external_size = max(new_external_size[0], MIN_SIZE), max(new_external_size[1], MIN_SIZE)
    external_image = Image.open(external_image_path)
    external_image_resized = external_image.resize(new_external_size, Image.LANCZOS)
    external_image_resized.save("test_cleaned.jpg")
else:
    print("Không có ảnh đã xử lý trong thư mục.")


In [212]:
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

model = timm.create_model('convnext_small_384_in22ft1k', pretrained=True)
model.eval()

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

/home/huy31/Projects/dog_emotions/venv/lib/python3.9/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_small_384_in22ft1k to current convnext_small.fb_in22k_ft_in1k_384.
  model = create_fn(


In [213]:
model.to('cpu')


ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)


In [5]:
def read_labels(fname):
    with open(fname, "r") as f:
        labels = [re.sub('[^0-9a-zA-Z ]+', " ", s.lower().strip()) for s in f.readlines()]
    return labels

In [214]:
categories = read_labels("imagenet_classes.txt")
dog_labels = read_labels("dog_labels.txt")


In [219]:
def detect_dog(img_file, transform):
    input_tensor = transform(Image.open(img_file))
    input_batch = input_tensor.unsqueeze(0)
    input_batch = input_batch.to('cpu')

    with torch.no_grad():
        output = model(input_batch)
    
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    top5_prob, top5_animal_id = torch.topk(probabilities, 5)
    animals = set([categories[c] for c in top5_animal_id])
    dog = animals.intersection(set(dog_labels))
        
    return len(dog) > 0, animals

In [220]:
detech_image = detect_dog('test_cleaned.jpg', transform)
print(detech_image)

(True, {'redbone', 'labrador retriever', 'bloodhound', 'rhodesian ridgeback', 'golden retriever'})


In [221]:
model = torch.load('dog_model.pth', map_location=torch.device('cpu'))

In [34]:
from torchvision.io import read_image
from torchvision import transforms as T
from torch.utils.data import Dataset
import math

class DogDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.augments = [T.RandomHorizontalFlip(1), T.RandomRotation(90), T.AutoAugment()]
        self.normalize = T.Compose([
            T.ConvertImageDtype(torch.float),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

    def __len__(self):
        return self.dataset.shape[0] * (len(self.augments) + 1)
    
    def classes(self):
        return self.dataset["code"].unique()

    def __getitem__(self, idx):
        augment = math.floor(idx / self.dataset.shape[0])
        idx = idx % self.dataset.shape[0]
        
        row = self.dataset.iloc[idx,:]
            
        img_path = row["filename"]
        image = read_image(img_path)
        
        label = row["code"]
        
        if augment > 0:
            image = self.augments[augment - 1].forward(image)
        
        image = self.normalize(image)

        return image, int(label), img_path

In [274]:
from PIL import Image
import pandas as pd

image_path = 'test_cleaned.jpg'
data = {'filename': [image_path], 'code': [0]}
image_df = pd.DataFrame(data)
image_set = DogDataset(image_df)
from torch.utils.data import DataLoader
data = DataLoader(image_set)

In [275]:
all_preds = list()
with torch.no_grad():
    for batch, (images, labels, img_paths) in enumerate(data):
        
        images = images.to(device)
        outputs = model(images.float())
        
        _, preds = torch.max(outputs.data, 1)
        all_preds.append(preds)
preds = list(np.concatenate([p.cpu().numpy() for p in all_preds]))

In [276]:
preds = list(np.concatenate([p.cpu().numpy() for p in all_preds]))
predictions = pd.DataFrame(dict(path=[image_path], happy=[preds.count(0) * 100 // len(preds)], relaxed=[preds.count(1) * 100 // len(preds)], sad=[preds.count(2) * 100 // len(preds)]))
response = f"""
Happy: {preds.count(0) * 100 / len(preds)} % 
Relaxed: {preds.count(1) * 100 / len(preds)} %
Sad: {preds.count(2) * 100 // len(preds)} %
"""
print(response)


Happy: 75.0 % 
Relaxed: 0.0 %
Sad: 25 %

